In [46]:
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import scale
import sklearn
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as AL
from sklearn.kernel_approximation import RBFSampler
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from threading import Thread
from time import time
from sklearn.metrics import accuracy_score
import plotly
import pandas as pd

## String Subsequence Kernel
This implementation uses dynamic programming to caculate the number of common string subsequences of length n of two strings, the distance between characters in the subsequence is penalized with a decay factor $\lambda$, the implementatin consist of n+1 iterations, where each iteration creates a matrix and each position j,k tells how many common subsequences of length i have been counted at the moment and wich decay the subsequece has.

In [47]:
def ssk (a, b, n, lam):
    Kp = np.zeros((n + 1, len(a), len(b)), dtype=np.float64)
    Kp[0, :] = 1.0
    for i in range(n):
        for j in range(len(a) - 1):
            Kpp = 0.0
            for k in range(len(b) - 1):
                Kpp = lam * (Kpp + lam * int(a[j] == b[k]) * Kp[i, j, k])           
                Kp[i+1, j+1, k+1] = lam * Kp[i+1, j, k+1] + Kpp
                
    K = 0.0
    for j in range(len(a)):
        for k in range(len(b)):
            K += lam * lam * int(a[j] == b[k]) * Kp[n-1, j, k]
    print(Kp)
    return  K

In [42]:
ssk("cvab", "cvag", 3, 1)

[[[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[0. 0. 0. 0.]
  [0. 1. 1. 1.]
  [0. 1. 2. 2.]
  [0. 1. 2. 3.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 1. 1.]
  [0. 0. 1. 3.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 1.]]]


1.0

## n-grams kernel 
### Implementation of the n-gram kernel taken from scikit-learn

In [43]:
def ngk(doc1, doc2, n=2):
    # Counts the occurences of unique n-grams
    ngrams = CountVectorizer(analyzer='char', ngram_range=(n, n)).fit_transform([doc1, doc2])

    # Normalize
    #a, b = TfidfTransformer().fit_transform(ngrams).toarray()
    a, b = ngrams.toarray()
    return np.dot(a, b)

## Load data

In [3]:
english = np.genfromtxt('words/english5000.txt',dtype='str')
spanish = np.genfromtxt('words/spanish5000.txt',dtype='str')
sl = np.zeros(spanish.shape)
el = np.full(spanish.shape, 1)
X = np.concatenate((spanish, english), axis=0)
Y = np.concatenate((sl, el), axis=0)

englishTest = np.genfromtxt('words/englishTest100.txt',dtype='str')
spanishTest = np.genfromtxt('words/spanishTest100.txt',dtype='str')
slt = np.zeros(spanishTest.shape)
elt = np.full(spanishTest.shape, 1)
XT = np.concatenate((spanishTest, englishTest), axis=0)
YT = np.concatenate((slt, elt), axis=0)

## SSK Classifier

In [5]:
gram = np.zeros((X.shape[0], X.shape[0]))
gramT = np.zeros((XT.shape[0], X.shape[0]))
def sskClassifier(n, lam):
    #t = time()
    for i in range(X.shape[0]):
        for j in range(X.shape[0]):
            gram[i][j] = ssk(X[i], X[j], n, lam)

    #print("gram preprocessed")
    #print(round(time()-t))

    clf = svm.SVC(kernel='precomputed')
    clf.fit(gram, Y)

    #t = time()
    for i in range(XT.shape[0]):
        for j in range(X.shape[0]):
            gramT[i][j] = ssk(XT[i], X[j], n, lam)

    #print("test gram preprocessed")
    #print(round(time()-t))
    Z = clf.predict(gramT)
    #print(Z)
    #print(YT)
    #print(XT)
    a = accuracy_score(YT, Z)
    print(a)
    return a

## Cross validation

In [7]:
nMax, lamMax, tmp = 0, 0, 0
for n in range(2, 8):
    for lam in range(1, 11):
        print("n ", n, "lamda ", lam/10.0)
        aux = sskClassifier(n, lam/10.0)
        if aux > tmp:
            tmp = aux
            nMax = n
            lamMax = lam

print("nMax", nMax, "lamMax", lamMax, "value", tmp)

n  2 lamda  0.1
0.715
n  2 lamda  0.2
0.72
n  2 lamda  0.3
0.705
n  2 lamda  0.4
0.695
n  2 lamda  0.5
0.745
n  2 lamda  0.6
0.77
n  2 lamda  0.7
0.775
n  2 lamda  0.8
0.73
n  2 lamda  0.9
0.74
n  2 lamda  1.0
0.725
n  3 lamda  0.1
0.59
n  3 lamda  0.2
0.585
n  3 lamda  0.3
0.61
n  3 lamda  0.4
0.665
n  3 lamda  0.5
0.705
n  3 lamda  0.6
0.75
n  3 lamda  0.7
0.745
n  3 lamda  0.8
0.76
n  3 lamda  0.9
0.72
n  3 lamda  1.0
0.71
n  4 lamda  0.1
0.61
n  4 lamda  0.2
0.65
n  4 lamda  0.3
0.67
n  4 lamda  0.4
0.62
n  4 lamda  0.5
0.565
n  4 lamda  0.6
0.555
n  4 lamda  0.7
0.735
n  4 lamda  0.8
0.69
n  4 lamda  0.9
0.685
n  4 lamda  1.0
0.65
n  5 lamda  0.1
0.525
n  5 lamda  0.2
0.515
n  5 lamda  0.3
0.505
n  5 lamda  0.4
0.5
n  5 lamda  0.5
0.5
n  5 lamda  0.6
0.5
n  5 lamda  0.7
0.615
n  5 lamda  0.8
0.66
n  5 lamda  0.9
0.625
n  5 lamda  1.0
0.62
n  6 lamda  0.1
0.5
n  6 lamda  0.2
0.5
n  6 lamda  0.3
0.5
n  6 lamda  0.4
0.5
n  6 lamda  0.5
0.5
n  6 lamda  0.6
0.5
n  6 lamda  0.7
0.5
n  6

### Best parameters:
### n= 2, lambda = 0.7

## NGK Classifier

## Load data

In [ ]:
english = np.genfromtxt('words/english4000.txt',dtype='str')
spanish = np.genfromtxt('words/spanish4000.txt',dtype='str')
sl = np.zeros(spanish.shape)
el = np.full(spanish.shape, 1)
X = np.concatenate((spanish, english), axis=0)
Y = np.concatenate((sl, el), axis=0)

englishTest = np.genfromtxt('words/englishTest100.txt',dtype='str')
spanishTest = np.genfromtxt('words/spanishTest100.txt',dtype='str')
slt = np.zeros(spanishTest.shape)
elt = np.full(spanishTest.shape, 1)
XT = np.concatenate((spanishTest, englishTest), axis=0)
YT = np.concatenate((slt, elt), axis=0)

In [9]:
def ngkClassifier(n):
    #t = time()
    gram2 = np.zeros((X.shape[0], X.shape[0]))
    for i in range(X.shape[0]):
        for j in range(X.shape[0]):
            gram2[i][j] = ngk(X[i], X[j], n)

    #print("gram2 preprocessed")
    #print(round(time()-t))

    clf2 = svm.SVC(kernel='precomputed')
    clf2.fit(gram2, Y)

    #t = time()
    gramT2 = np.zeros((XT.shape[0], X.shape[0]))
    for i in range(XT.shape[0]):
        for j in range(X.shape[0]):
            gramT2[i][j] = ngk(XT[i], X[j], n)

    #print("test gram2 preprocessed")
    #print(round(time()-t))

    Z2 = clf2.predict(gramT2)
    #print(Z2)
    #print(YT)
    #print(XT)
    a = accuracy_score(YT, Z2)
    print(a)
    return a

(198, 198)
gram2 preprocessed
145
(20, 198)
test gram2 preprocessed
16
[1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
['analizado' 'ambientes' 'amaban' 'almanaque' 'alisa' 'alimenticia'
 'alimentado' 'alcancé' 'albóndigas' 'albin' 'freshet' 'grovel'
 'harlequin' 'jewry' 'megaphone' 'parallelogram' 'plummet' 'sublunary'
 'typographical' 'ussr']


## Cross validation

In [5]:
nMax, tmp = 0, 0
for n in range(1, 5):
    print("n ", n)
    aux = ngkClassifier(n)
    if aux > tmp:
        tmp = aux
        nMax = n
        
print("Max n:", nMax, "value:", tmp)


n  1
0.715
n  2
0.785
n  3
0.71
n  4
0.695
Max n: 2 value: 0.785


In [6]:
english = np.genfromtxt('words/english4000.txt',dtype='str')
spanish = np.genfromtxt('words/spanish4000.txt',dtype='str')
sl = np.zeros(spanish.shape)
el = np.full(spanish.shape, 1)
X = np.concatenate((spanish, english), axis=0)
Y = np.concatenate((sl, el), axis=0)

englishTest = np.genfromtxt('words/englishTest100.txt',dtype='str')
spanishTest = np.genfromtxt('words/spanishTest100.txt',dtype='str')
slt = np.zeros(spanishTest.shape)
elt = np.full(spanishTest.shape, 1)
XT = np.concatenate((spanishTest, englishTest), axis=0)
YT = np.concatenate((slt, elt), axis=0)

In [ ]:
a = sskClassifier(2, 0.7)